In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import matplotlib.pyplot as plt

In [ ]:
# Define paths
dataset_dir = "D://SEM-4//PROJECTS//ML//DATASET//Archive"  # replace with the path to your dataset directory

In [ ]:

# Image data generators with validation split
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.25,  # 25% for validation/testing
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [2]:
# Training generator
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 8738 images belonging to 13 classes.


In [3]:
# Validation generator
val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 2909 images belonging to 13 classes.


In [4]:
# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [5]:
# Freeze the layers of VGG16 model
for layer in base_model.layers:
    layer.trainable = False

In [6]:
# Feature extraction
x = Flatten()(base_model.output)

In [7]:
# Define the model building function for Keras Tuner
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(base_model)
    model.add(Flatten())
    
    # Tune the number of units in the Dense layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=128, max_value=512, step=64),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout_' + str(i), min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(13, activation='softmax'))  # 13 classes

    # Tune the learning rate for the optimizer
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [8]:
# Instantiate the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=12,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='image_classification'
)

Reloading Tuner from tuner_dir\image_classification\tuner0.json


In [9]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [10]:
# Search for the best hyperparameters
tuner.search(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=50,  # max epochs
    callbacks=[early_stopping]
)

Trial 12 Complete [01h 10m 54s]
val_accuracy: 0.24137930572032928

Best val_accuracy So Far: 0.48275861144065857
Total elapsed time: 19h 16m 30s


In [11]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [12]:
# Build the best model
model = tuner.hypermodel.build(best_hps)

In [13]:
# Train the model with the optimal hyperparameters
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=50,  # you can adjust the number of epochs
    callbacks=[early_stopping]
)

Epoch 1/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2280 - loss: 2.3462

C:\Users\garik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


273/273 ━━━━━━━━━━━━━━━━━━━━ 885s 3s/step - accuracy: 0.2282 - loss: 2.3456 - val_accuracy: 0.2344 - val_loss: 2.3220
Epoch 2/50
  1/273 ━━━━━━━━━━━━━━━━━━━━ 6:56 2s/step - accuracy: 0.2812 - loss: 2.0292

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


273/273 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2812 - loss: 2.0292 - val_accuracy: 0.3103 - val_loss: 2.4210
Epoch 3/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 589s 2s/step - accuracy: 0.3555 - loss: 1.9464 - val_accuracy: 0.2622 - val_loss: 2.2585
Epoch 4/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3750 - loss: 1.8321 - val_accuracy: 0.4483 - val_loss: 1.8485
Epoch 5/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 588s 2s/step - accuracy: 0.3910 - loss: 1.8378 - val_accuracy: 0.2326 - val_loss: 2.3060
Epoch 6/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5625 - loss: 1.6179 - val_accuracy: 0.1724 - val_loss: 2.4059
Epoch 7/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 583s 2s/step - accuracy: 0.4231 - loss: 1.7518 - val_accuracy: 0.2618 - val_loss: 2.2598
Epoch 8/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5625 - loss: 1.3234 - val_accuracy: 0.3448 - val_loss: 1.8018
Epoch 9/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 589s 2s/step - accuracy: 0.4422 - loss: 1.6946 - val_accuracy: 0.2729 - 

In [ ]:
# Print training and validation accuracy for each epoch
for epoch, (train_acc, val_acc) in enumerate(zip(history.history['accuracy'], history.history['val_accuracy'])):
    print(f'Epoch {epoch + 1}: Training Accuracy: {train_acc * 100:.2f}%, Validation Accuracy: {val_acc * 100:.2f}%')

In [ ]:
# Get the final training accuracy
final_training_accuracy = history.history['accuracy'][-1]
print(f'Final Training Accuracy: {final_training_accuracy * 100:.2f}%')

In [ ]:
# You can also get the final validation accuracy
final_validation_accuracy = history.history['val_accuracy'][-1]
print(f'Final Validation Accuracy: {final_validation_accuracy * 100:.2f}%')

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
plt.show()

In [14]:
# Evaluate the model on the test set
test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2909 images belonging to 13 classes.


In [15]:

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Loss: {test_loss:.4f}')

C:\Users\garik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


90/90 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.3426 - loss: 2.0055
Test Accuracy: 26.63%
Test Loss: 2.2492
